In [ ]:
'''
from google.colab import drive
drive.mount('/content/drive')
df = pd.read_csv("/content/drive/MyDrive/code/chennai_weather_processed (2).csv")'''

In [ ]:
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objs as go
from plotly.subplots import make_subplots

CSV_FILE = 'chennai_weather_processed (2).csv'
df_raw = pd.read_csv(CSV_FILE, parse_dates=['date'])
df_raw.set_index('date', inplace=True)

daily = pd.DataFrame({
    'temp':   df_raw['temperature_2m'].resample('D').mean(),
    'precip': df_raw['prcp'].resample('D').sum()
}).dropna()

split = int(len(daily) * 0.9)
train = daily.iloc[:split]
test  = daily.iloc[split:]

model_t    = ARIMA(train['temp'], order=(5,1,0)).fit()
forecast_t = model_t.forecast(steps=len(test))


In [ ]:

# 5. Interactive Plotly: Temperature Forecast
fig1 = go.Figure()
fig1.add_trace(go.Scatter(x=train.index, y=train['temp'], mode='lines', name='Train (Temp)'))
fig1.add_trace(go.Scatter(x=test.index,  y=test['temp'],  mode='lines', name='Actual (Temp)', line=dict(color='green')))
fig1.add_trace(go.Scatter(x=test.index,  y=forecast_t,    mode='lines', name='Forecast (Temp)', line=dict(color='red')))
fig1.update_layout(title='ARIMA Forecast — Daily Avg Temperature',
                  xaxis_title='Date', yaxis_title='°C', hovermode='x unified')
fig1.show()


In [ ]:

# 6. Fit ARIMA on precipitation and forecast
model_p    = ARIMA(train['precip'], order=(2,0,2)).fit()
forecast_p = model_p.forecast(steps=len(test))

# 7. Interactive Plotly: Precipitation Forecast
fig2 = go.Figure()
fig2.add_trace(go.Scatter(x=train.index, y=train['precip'], mode='lines', name='Train (Precip)'))
fig2.add_trace(go.Scatter(x=test.index,  y=test['precip'],  mode='lines', name='Actual (Precip)', line=dict(color='blue')))
fig2.add_trace(go.Scatter(x=test.index,  y=forecast_p,      mode='lines', name='Forecast (Precip)', line=dict(color='orange')))
fig2.update_layout(title='ARIMA Forecast — Daily Total Precipitation',
                   xaxis_title='Date', yaxis_title='mm', hovermode='x unified')
fig2.show()


In [ ]:
# 8. Combined subplot (temperature + precipitation)
fig3 = make_subplots(rows=2, cols=1, shared_xaxes=True, vertical_spacing=0.1,
                     subplot_titles=('Temperature (°C)', 'Precipitation (mm)'))

# Temperature subplot
fig3.add_trace(go.Scatter(x=train.index, y=train['temp'], name='Train (Temp)'), row=1, col=1)
fig3.add_trace(go.Scatter(x=test.index,  y=test['temp'],  name='Actual (Temp)', line=dict(color='green')), row=1, col=1)
fig3.add_trace(go.Scatter(x=test.index,  y=forecast_t,    name='Forecast (Temp)', line=dict(color='red')), row=1, col=1)

# Precipitation subplot
fig3.add_trace(go.Scatter(x=train.index, y=train['precip'], name='Train (Precip)'), row=2, col=1)
fig3.add_trace(go.Scatter(x=test.index,  y=test['precip'],  name='Actual (Precip)', line=dict(color='blue')), row=2, col=1)
fig3.add_trace(go.Scatter(x=test.index,  y=forecast_p,      name='Forecast (Precip)', line=dict(color='orange')), row=2, col=1)

fig3.update_layout(title='ARIMA Forecasts — Temperature & Precipitation',
                   hovermode='x unified', height=700, showlegend=True)
fig3.update_xaxes(title_text='Date', row=2, col=1)
fig3.update_yaxes(title_text='°C', row=1, col=1)
fig3.update_yaxes(title_text='mm', row=2, col=1)
fig3.show()


In [ ]:
# Create a 7-day forecast plot with vertical line for current date
import pandas as pd
import numpy as np
from statsmodels.tsa.arima.model import ARIMA
import plotly.graph_objs as go
from datetime import datetime, timedelta

# Assuming you've already loaded and processed your data as in your original code
# Use the full dataset for the model
model_full = ARIMA(daily['temp'], order=(5,1,0)).fit()

# Set up forecast for next 7 days
future_steps = 7
future_temp = model_full.forecast(steps=future_steps)

# Create date range for forecast (starting from current date)
current_date = datetime.now().replace(hour=0, minute=0, second=0, microsecond=0)
forecast_dates = pd.date_range(start=current_date, periods=future_steps+1)  # Include current date

# Create data for plotting (including current day's temperature)
# For current day, use either actual data or forecast
current_temp = daily['temp'].iloc[-1] if current_date.date() <= daily.index[-1].date() else future_temp[0]
plot_temps = np.concatenate(([current_temp], future_temp))

# Create the figure
fig = go.Figure()

# Add the temperature line
fig.add_trace(go.Scatter(
    x=forecast_dates,
    y=plot_temps,
    mode='lines+markers',
    line=dict(color='red', width=2),
    name='Forecast'
))

# Add vertical line for tomorrow (first forecasted day)
tomorrow = current_date + timedelta(days=1)
fig.add_vline(
    x=tomorrow,
    line=dict(color='black', dash='dash', width=1)
)

# Add annotation for temperature at the vertical line
tomorrow_temp = future_temp[0]
fig.add_annotation(
    x=tomorrow,
    y=tomorrow_temp,
    text=f"Apr {tomorrow.day}, {tomorrow.year}<br>Forecast (Next 7 Days) : Date: Apr {tomorrow.day}, {tomorrow.year}<br>Temp: {tomorrow_temp:.2f}°C",
    showarrow=True,
    arrowhead=1,
    ax=0,
    ay=-40,
    bgcolor="white",
    bordercolor="black",
    borderwidth=1
)

# Update layout
fig.update_layout(
    title="Forecasted Daily Avg Temperature for Next 7 Days (ARIMA)",
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    xaxis=dict(tickformat="%b %d<br>%Y"),
    hovermode="x unified",
    template="plotly_white"
)

# Adjust y-axis range to match the style in the image
y_min = min(plot_temps) - 0.1
y_max = max(plot_temps) + 0.1
fig.update_yaxes(range=[y_min, y_max])

fig.show()


<ipython-input-31-0605b967d6b1>:45: FutureWarning:

Series.__getitem__ treating keys as positions is deprecated. In a future version, integer keys will always be treated as labels (consistent with DataFrame behavior). To access a value by position, use `ser.iloc[pos]`



In [ ]:
import pandas as pd
import plotly.graph_objects as go
from statsmodels.tsa.arima.model import ARIMA

daily_temp = df_raw['temperature_2m'].resample('D').mean().dropna()
daily_prcp = df_raw['prcp'].resample('D').sum().dropna()

# 2) Fit ARIMA on full daily series
model_temp = ARIMA(daily_temp, order=(5,1,0)).fit()
model_prcp = ARIMA(daily_prcp, order=(2,0,2)).fit()


# 4) Classification functions
def classify_temp_dynamic(val):
    if val <= temp_low:   return 'Low'
    if val <= temp_high:  return 'Moderate'
    return 'High'

def classify_weather_risk(temp, prcp):
    if prcp >= prcp_severe:  return "Severe: Heavy rain expected"
    if prcp >= prcp_high:    return "Low Risk"
    if temp >= 40:           return "Very High heat expected"
    if temp >= 35:           return "High heat expected"
    if temp >= 28:           return "Moderate heat expected"
    if temp <= 5:            return "Cold weather expected"
    if temp <= 15:           return "Cool weather expected"
    return "Mild weather expected"

# 5) Forecast + risk plotting
def plot_risk_forecast(start_date):
    days = 7
    future_temp = model_temp.forecast(steps=days)
    future_prcp = model_prcp.forecast(steps=days)
    future_dates = pd.date_range(start=start_date, periods=days)

    fc = pd.DataFrame({
        'Date':   future_dates,
        'Temp':   future_temp.values,
        'Precip': future_prcp.values
    })
    fc['Temp_Level'] = fc['Temp'].apply(classify_temp_dynamic)
    fc['Risk']       = [classify_weather_risk(t, p) for t, p in zip(fc['Temp'], fc['Precip'])]

    # Plot: temperature line + precipitation bar
    fig = go.Figure()
    fig.add_trace(go.Scatter(
        x=fc['Date'], y=fc['Temp'],
        mode='lines+markers', name='Temperature (°C)'
    ))
    fig.add_trace(go.Bar(
        x=fc['Date'], y=fc['Precip'],
        name='Precipitation (mm)', opacity=0.5, yaxis='y2'
    ))
    fig.update_layout(
        title="7‑Day Forecast & Risk Analysis",
        xaxis_title="Date",
        yaxis=dict(title="Temperature (°C)", side='left'),
        yaxis2=dict(title="Precipitation (mm)", overlaying='y', side='right'),
        hovermode="x unified",
        legend=dict(x=0.01, y=0.99),
        template="plotly_white"
    )
    fig.show()

    # Display forecast and risk table
    print("Forecast with Risk Levels:")
    display(fc[['Date','Temp','Temp_Level','Risk']])

# 6) Run for the next day after your last date
start_date = daily_temp.index[-1] + pd.Timedelta(days=1)
plot_risk_forecast(start_date)

Forecast with Risk Levels:


,Date,Temp,Temp_Level,Risk
0,2025-04-23,27.800116,Moderate,Low Risk
1,2025-04-24,28.562353,Moderate,Low Risk
2,2025-04-25,28.994411,Moderate,Low Risk
3,2025-04-26,29.096529,Moderate,Low Risk
4,2025-04-27,29.198646,Moderate,Low Risk
5,2025-04-28,28.986296,Moderate,Low Risk
6,2025-04-29,28.880622,Moderate,Low Risk


In [ ]:
import pandas as pd
import plotly.graph_objects as go
from plotly.subplots import make_subplots
from statsmodels.tsa.arima.model import ARIMA

# 1) Load & resample to daily temperature
CSV_PATH = 'chennai_weather_processed (2).csv'
df = pd.read_csv(CSV_PATH, parse_dates=['date']).set_index('date')
daily_temp = df['temperature_2m'].resample('D').mean().dropna()

# 2) Train/test split (90% train, 10% test)
split = int(len(daily_temp) * 0.9)
train_temp = daily_temp.iloc[:split]
test_temp  = daily_temp.iloc[split:]

# 3) Fit ARIMA & forecast on test temperature
model_t = ARIMA(train_temp, order=(5,1,0)).fit()
fc_t    = model_t.forecast(steps=len(test_temp))

# 4) Select only the last 30 days of the test period
recent_start = test_temp.index.max() - pd.Timedelta(days=30)
rec_true_t   = test_temp.loc[recent_start:]
rec_fc_t     = fc_t.loc[recent_start:]

# 5) Build interactive Plotly figure for temperature only
fig = go.Figure()

fig.add_trace(go.Scatter(
    x=rec_true_t.index, y=rec_true_t.values,
    mode='lines+markers',
    name='Actual Temp (°C)',
    marker=dict(symbol='circle')
))
fig.add_trace(go.Scatter(
    x=rec_true_t.index, y=rec_fc_t.values,
    mode='lines+markers',
    name='Forecast Temp (°C)',
    marker=dict(symbol='square')
))

# 6) Layout adjustments
fig.update_layout(
    title_text="Interactive ARIMA Forecast vs Actual Temperature (Last 30 Days)",
    xaxis_title="Date",
    yaxis_title="Temperature (°C)",
    legend=dict(orientation="h", yanchor="bottom", y=1.02, xanchor="right", x=1),
    margin=dict(l=50, r=50, t=80, b=50),
    template="plotly_white"
)

# 7) Show plot
fig.show()